In [56]:
import requests
import re
import mysql.connector
import pandas as pd
mydb = mysql.connector.connect(
  host="163.44.196.192",
  user="ararize",
  password="isylzjko$3rv3r",
  database="tp"
)
mycursor = mydb.cursor()


In [57]:
def request_BoilerMea():
    sql = 'SELECT `meID`,`bcL1picName`,`bcL2picName`,`bcL3picName`,`bcL4picName`,`bcL5picName`,`bcL6picName` FROM `boiler_data` WHERE bdType = 3'
    mycursor.execute(sql)
    myresult = mycursor.fetchall()
    boilerMea_data = pd.DataFrame(myresult, columns = ['meID','meaL1','meaL2','meaL3','meaL4','meaL5','meaL6'])
    boilerMea_data['meID'] = boilerMea_data['meID'].astype('str')
    return boilerMea_data
def request_BoilerID():
    sql = "SELECT * FROM `boilerID`"
    mycursor.execute(sql)
    myresult = mycursor.fetchall()
    boilerID_data = pd.DataFrame(myresult, columns = ['meID','BoilerID'])
    boilerID_data['meID'] = boilerID_data['meID'].astype('str')
    return boilerID_data
def mapMea(mea):
    mea = mea.str.replace("\"","").str.replace("[","").str.replace("]","").str.replace("0","")
    mea_id = pd.read_csv('mea_id.csv',dtype={'meaID':'str'})
    mea_id = dict(zip(mea_id['meaID'],mea_id['meaName']))
    for j in range(9):
        mea = mea.str.replace(str(j+1),mea_id[str(j+1)])
    return mea

boilerMea_data = request_BoilerMea()
boilerID_data = request_BoilerID()
for i in range(6):
    boilerMea_data.iloc[:,i+1] = mapMea(boilerMea_data.iloc[:,i+1])
df_mea = boilerID_data.merge(boilerMea_data,how='left',on='meID')

In [60]:
df_mea.drop(['meID'],axis=1).set_index('BoilerID').to_csv('mea_Data.csv')